## Application

In [ ]:
# Define app.
api = FastAPI(
    title="doggos", 
    description="classify your dog", 
    version="0.1",
)


In [ ]:
@serve.deployment
@serve.ingress(api)
class Doggos:
    def __init__(self, classifier):
        self.classifier = classifier
        
    @api.post("/predict/")
    async def predict(self, request: Request):
        data = await request.json()
        probabilities = await self.classifier.get_probabilities.remote(url=data["url"])
        return probabilities


In [ ]:
# Model registry.
model_registry = "/mnt/cluster_storage/mlflow/doggos"
experiment_name = "doggos"
mlflow.set_tracking_uri(f"file:{model_registry}")


In [ ]:
# Get best_run's artifact_dir.
sorted_runs = mlflow.search_runs(
    experiment_names=[experiment_name], 
    order_by=["metrics.val_loss ASC"])
best_run = sorted_runs.iloc[0]
artifacts_dir = urlparse(best_run.artifact_uri).path


In [ ]:
# Define app.
app = Doggos.bind(
    classifier=ClassPredictor.bind(
        model_id="openai/clip-vit-base-patch32",
        artifacts_dir=artifacts_dir,
        device="cuda"
    )
)


In [ ]:
# Run service locally.
serve.run(app, route_prefix="/")


2025-08-22 00:51:12,070	INFO worker.py:1747 -- Connecting to existing Ray cluster at address: 10.0.52.10:6379...
2025-08-22 00:51:12,082	INFO worker.py:1918 -- Connected to Ray cluster. View the dashboard at https://session-466hy7cqu1gzrp8zk8l4byz7l7.i.anyscaleuserdata.com 
2025-08-22 00:51:12,091	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_bb1ea558d334a00804951688d29c76ff051341cc.zip' (1.11MiB) to Ray cluster...
2025-08-22 00:51:12,096	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_bb1ea558d334a00804951688d29c76ff051341cc.zip'.
INFO 2025-08-22 00:51:12,153 serve 133557 -- Connecting to existing Serve app in namespace "serve". New http options will not be applied.
(ServeController pid=61167) INFO 2025-08-22 00:51:12,275 controller 61167 -- Deploying new version of Deployment(name='ClassPredictor', app='default') (initial target replicas: 1).
(ServeController pid=61167) INFO 2025-08-22 00:51:12,276 controller 61167 -- Deploying new version of

DeploymentHandle(deployment='Doggos')

In [ ]:
# Send a request.
url = "https://doggos-dataset.s3.us-west-2.amazonaws.com/samara.png"
data = {"url": url}
response = requests.post("http://127.0.0.1:8000/predict/", json=data)
probabilities = response.json()["probabilities"]
sorted_probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)
sorted_probabilities[0:3]


(ServeReplica:default:ClassPredictor pid=30024, ip=10.0.4.102) /home/ray/anaconda3/lib/python3.12/site-packages/ray/serve/_private/replica.py:1376: UserWarning: Calling sync method 'get_probabilities' directly on the asyncio loop. In a future version, sync methods will be run in a threadpool by default. Ensure your sync methods are thread safe or keep the existing behavior by making them `async def`. Opt into the new behavior by setting RAY_SERVE_RUN_SYNC_IN_THREADPOOL=1.
(ServeReplica:default:ClassPredictor pid=30024, ip=10.0.4.102)   warnings.warn(
(ServeReplica:default:Doggos pid=133722) INFO 2025-08-22 00:51:22,490 default_Doggos 0qpk1kw9 daf11861-073e-4758-82b7-0b222066b668 -- Started <ray.serve._private.router.SharedRouterLongPollClient object at 0x72f86c539670>.


[('collie', 0.2292557954788208),
 ('border_collie', 0.1228194534778595),
 ('german_shepherd', 0.07383470982313156)]

(ServeReplica:default:ClassPredictor pid=30024, ip=10.0.4.102) INFO 2025-08-22 00:51:23,011 default_ClassPredictor qtsnu3yv daf11861-073e-4758-82b7-0b222066b668 -- CALL /predict/ OK 504.9ms
(ServeReplica:default:Doggos pid=133722) INFO 2025-08-22 00:51:23,013 default_Doggos 0qpk1kw9 daf11861-073e-4758-82b7-0b222066b668 -- POST /predict/ 200 567.1ms


(autoscaler +13m35s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
